In [21]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [28]:
num_encoder_tokens = 5000
num_decoder_tokens = 5000
latent_dim = 128

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs) 
encoder_states = [state_h, state_c] 
decoder_inputs = Input(shape=(None, num_decoder_tokens)) 
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model1 = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model1.summary()

from keras.utils import plot_model
plot_model(model1, to_file='model_png/202_1Seq2seq_train.png', show_layer_names=True, show_shapes=True) 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, None, 5000)   0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, None, 5000)   0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, 128), (None, 2626048     input_31[0][0]                   
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, None, 128),  2626048     input_32[0][0]                   
                                                                 lstm_10[0][1]                    
          

In [30]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
decoder_model.summary()

from keras.utils import plot_model
plot_model(encoder_model, to_file='model_png/202_2Seq2seq_p_encoder.png', show_layer_names=True, show_shapes=True) 
plot_model(decoder_model, to_file='model_png/202_2Seq2seq_p_decoder.png', show_layer_names=True, show_shapes=True) 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, None, 5000)   0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, None, 128),  2626048     input_32[0][0]                   
                                                                 input_35[0][0]                   
          